In [1]:
%env XDG_CACHE=/workspace/.cache
%env HF_HOME=/workspace/.cache/huggingface


env: XDG_CACHE=/workspace/.cache
env: HF_HOME=/workspace/.cache/huggingface


In [2]:
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList
from datasets import load_dataset
import pyonmttok
import ctranslate2
from metrics import *


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
model_id = "projecte-aina/aguila-7b"
#model_id = "tiiuae/falcon-7b"
model_name = model_id.split('/')[1]
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True,
                                             device_map="auto")


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
## Lets Do the translation layer
from huggingface_hub import snapshot_download
print("Loading translator Models...")

ca_en_model_folder = snapshot_download(repo_id="projecte-aina/mt-aina-ca-en", revision="main")
tokenizer_ca_en = pyonmttok.Tokenizer(
    mode="none", sp_model_path=ca_en_model_folder + "/spm.model"
)
ca_en_model = ctranslate2.Translator(ca_en_model_folder, device="cuda")


Loading translator Models...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
class StopTextCriteria(StoppingCriteria):
    def __init__(self, stop_text, tokenizer):
        self.stop_text_id = tokenizer.encode(stop_text, add_special_tokens=False)
        self.stop_text_length = len(self.stop_text_id)
        self.batch_size = None
        self.stop_track = None

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs):
        if self.stop_track is None:
            self.stop_track = {i for i in range(input_ids.shape[0])}

        # Check if the end of the input_ids matches the stop_text_id
        remove = []
        for i in self.stop_track:
            if input_ids[i, -self.stop_text_length:].tolist() == self.stop_text_id:
                remove.append(i)

        [self.stop_track.remove(i) for i in remove]

        if len(self.stop_track) == 0:
            return True
        else:
            return False


def run_inference(sequences, num_tokens=20, stop_text='\n'):
    tokens = tokenizer(sequences, return_tensors="pt", padding=True).to(model.device)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    stop_criteria = None
    if stop_text:
        stop_criteria = StoppingCriteriaList([StopTextCriteria(stop_text, tokenizer)])

    with torch.no_grad():
        # Generate tokens
        input_ids = model.generate(input_ids,
                                    attention_mask=attention_mask,
                                    do_sample=True,
                                    top_k=1,
                                    eos_token_id=tokenizer.eos_token_id,
                                    stopping_criteria=stop_criteria,
                                    max_new_tokens=num_tokens)

        # Concatenate along the second dimension
        ones_tensor = torch.ones(attention_mask.size(0), 1, dtype=attention_mask.dtype).to(model.device)
        attention_mask = torch.cat((attention_mask, ones_tensor), dim=1)

        # Decode the generated tokens into text
        generated_sequences = tokenizer.batch_decode(input_ids, attention_mask=attention_mask, skip_special_tokens=True)

        res = []
        for i, seq in enumerate(generated_sequences):
            generated = seq.replace(sequences[i],  "")
            if stop_text:
                generated = generated.split(stop_text)[0]
            res.append(generated)

        return res


def translate(sample):
    def translate_to_english(txt):
        lines = [l for l in txt.split("\n") if l.strip() != ""]

        toks, _ = tokenizer_ca_en.tokenize_batch(lines)
        translated = ca_en_model.translate_batch(toks)
        ts = []
        for t in translated:
            t_str = tokenizer_ca_en.detokenize(t.hypotheses[0])
            # That is a bug on the translation outputing twice the translation.
            if len(txt.split(" ")) == 1 and len(t_str.split(" ")) == 2:
                t_str = t_str.split(" ")[0]
            ts.append(t_str)

        return "\n".join(ts)
    en_prompt = translate_to_english(sample['prompt'])
    en_answer = translate_to_english(sample['answer'])
    return {"prompt": en_prompt, "answer": en_answer}


def compute_metrics(samples):
    predictions = run_inference(samples['prompt'], stop_text=None)
    f1 = []
    for p, sample in zip(predictions, samples['answer']):
        f1.append(f1_score(p, sample))

    return {"f1": f1}


# Eval catalanqa

In [6]:
catalanqa = load_dataset("data", data_files="catalanqa.csv", split="train[:100]")


def compute_metrics(samples):
    predictions = run_inference(samples['prompt'], stop_text="\n")
    f1 = []
    for p, sample in zip(predictions, samples['answer']):
        f1.append(f1_score(p, sample))

    return {"f1": f1}

catalanqa.map(compute_metrics, batched=True, batch_size=64)
# catalanqa_en = catalanqa.map(translate)
# results_catalanqa_ca = catalanqa.map(compute_metrics)
# results_calalanqa_en = catalanqa_en.map(compute_metrics)

# from pathlib import Path
# Path("results").mkdir(parents=True, exist_ok=True)

# results_catalanqa_ca.to_csv(f"results/{model_name}-catalanqa-ca.csv", index=False)
# results_calalanqa_en.to_csv(f"results/{model_name}-catalanqa-en.csv", index=False)


Parameter 'function'=<function compute_metrics at 0x7f4778838ca0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/workspace/poligloteval/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


KeyboardInterrupt: 

# Eval xquad


In [ ]:
xquad_ca = load_dataset("data", data_files="xquad_ca.csv", split="train")
xquad_en = load_dataset("data", data_files="xquad_en.csv", split="train")

results_ca = xquad_ca.map(compute_metrics)
results_en = xquad_en.map(compute_metrics)

from pathlib import Path
Path("results").mkdir(parents=True, exist_ok=True)

results_ca.to_csv(f"results/{model_name}-xquad-ca.csv", index=False)
results_en.to_csv(f"results/{model_name}-xquad-en.csv", index=False)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/workspace/poligloteval/venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/workspace/poligloteval/venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/workspace/poligloteval/venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-g

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

18476

In [ ]:
del model
del tokenizer
torch.cuda.empty_cache()

model_id = "tiiuae/falcon-7b"
model_name = model_id.split('/')[1]
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.bfloat16,
                                             trust_remote_code=True,
                                             device_map="auto")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Eval catalanqa

In [ ]:
results_catalanqa_ca = catalanqa.map(compute_metrics)
results_calalanqa_en = catalanqa_en.map(compute_metrics)

from pathlib import Path
Path("results").mkdir(parents=True, exist_ok=True)

results_catalanqa_ca.to_csv(f"results/{model_name}-catalanqa-ca.csv", index=False)
results_calalanqa_en.to_csv(f"results/{model_name}-catalanqa-en.csv", index=False)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The atte

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The atte

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

10564

# Eval xquad


In [ ]:
results_ca = xquad_ca.map(compute_metrics)
results_en = xquad_en.map(compute_metrics)

from pathlib import Path
Path("results").mkdir(parents=True, exist_ok=True)

results_ca.to_csv(f"results/{model_name}-xquad-ca.csv", index=False)
results_en.to_csv(f"results/{model_name}-xquad-en.csv", index=False)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The atte

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The atte

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

18429